## Download shapefile from Natural Earth Data

Use curl to download the 1:110m Cultural Vectors file from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/

In [2]:
# !curl -L 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip' -o './raw_data/map/ne_110m_admin_0_countries.zip'

Next, unzip the archive to extract the shapefile (.shp), and some other junk:

In [3]:
# !unzip -o './raw_data/map/ne_110m_admin_0_countries.zip' -d './raw_data/map'

Edit the shapefile manually in QGIS.

Combine China and Taiwan. 

Save at:
`../raw_data/map/taiwan-merge/taiwan-merged.shp`

Convert shapefile to geojson

In [2]:
# npm install -g shapefile

!shp2json '../raw_data/map/taiwan-merge/taiwan-merged.shp' -o '../raw_data/map/countries.json'

Split up the geojson file so that each line ends with a `\n` character

In [29]:
# npm install -g ndjson-cli

!ndjson-split 'd.features' \
    < '../raw_data/map/countries.json' \
    > '../raw_data/map/countries.ndjson'

In [30]:
!wc -l ../raw_data/map/countries.ndjson

     176 ../raw_data/map/countries.ndjson


In [32]:
!rm ../raw_data/map/countries.json

Add a Code key for each country, so that it can be used for the join and reduce the amount of properties for each country, to just the name `NAME_EN`, country code `ADM0_A3`, altcode `GU_A3` - this has been edited in qgis - and region `REGION_UN`.

In [33]:
!ndjson-map 'd.code = d.properties.ADM0_A3, d' < ../raw_data/map/countries.ndjson \
    | ndjson-map 'd.properties = { name:d.properties.NAME_EN, code:d.properties.ADM0_A3, altcode:d.properties.GU_A3, region:d.properties.REGION_UN }, d' \
    > ../raw_data/map/temp.ndjson


In [34]:
!rm ../raw_data/map/countries.ndjson

In [35]:
!mv ../raw_data/map/temp.ndjson ../raw_data/map/countries.ndjson

## Join the measles vaccination and cases data with the map

Turn the vaccinaiton and cases csv files into ndjson files

-d, --dynamic-typing
    Convert booleans and numeric to their type instead of strings.

In [36]:
!csv2json -d \
    < ../raw_data/cases/cases-by-who-region-2019.csv \
    > ../raw_data/map/case-by-who-region-2019.json

The resulting file is a JSON array. To convert it to an NDJSON stream, use ndjson-cat (to remove the newlines), ndjson-split (to separate the array into multiple lines). You can run these individually, but here’s how to do it all in one go:

In [37]:
!ndjson-cat ../raw_data/map/case-by-who-region-2019.json \
  | ndjson-split \
  > ../raw_data/map/case-by-who-region-2019.ndjson

In [38]:
!rm ../raw_data/map/case-by-who-region-2019.json

In [39]:
!csv2json -d \
    < ../raw_data/vaccination/vaccination-by-who-region-2019.csv \
    > ../raw_data/map/vaccination-by-who-region-2019.json

In [40]:
!ndjson-cat ../raw_data/map/vaccination-by-who-region-2019.json \
  | ndjson-split \
  > ../raw_data/map/vaccination-by-who-region-2019.ndjson

In [41]:
!rm ../raw_data/map/vaccination-by-who-region-2019.json

Join the vaccination and cases data to the world map data. Make this a left outer join, so that each country is kept, even if they don't have any malaria data.

In [42]:
!ndjson-join --left 'd.code' \
    ../raw_data/map/countries.ndjson \
    ../raw_data/map/case-by-who-region-2019.ndjson \
    > ../raw_data/map/temp.ndjson

In [43]:
!rm ../raw_data/map/countries.ndjson
!mv ../raw_data/map/temp.ndjson ../raw_data/map/countries.ndjson

This results in a file with two json objects per line. Add the second object as a 'cases' to the first objects 'properties' To get rid of the second object use ndjson-map and Object.assign

In [44]:
!ndjson-map 'd[0].properties.cases = d[1], d' < ../raw_data/map/countries.ndjson \
    | ndjson-map 'Object.assign(d[0])' \
    > ../raw_data/map/temp.ndjson

In [45]:
!rm ../raw_data/map/countries.ndjson
!mv ../raw_data/map/temp.ndjson ../raw_data/map/countries.ndjson

Do the same with the vaccination data

In [46]:
!ndjson-join --left 'd.code' \
    ../raw_data/map/countries.ndjson \
    ../raw_data/map/vaccination-by-who-region-2019.ndjson \
    > ../raw_data/map/temp.ndjson

In [47]:
!rm ../raw_data/map/countries.ndjson
!mv ../raw_data/map/temp.ndjson ../raw_data/map/countries.ndjson

In [48]:
!ndjson-map 'd[0].properties.vaccination = d[1], d' < ../raw_data/map/countries.ndjson \
    | ndjson-map 'Object.assign(d[0])' \
    > ../raw_data/map/temp.ndjson

In [49]:
!rm ../raw_data/map/countries.ndjson
!mv ../raw_data/map/temp.ndjson ../raw_data/map/countries.ndjson

## Convert back from ndjson

In [50]:
!ndjson-reduce < ../raw_data/map/countries.ndjson \
    | ndjson-map '{type: "FeatureCollection", features: d}' \
    > ../raw_data/map/countries.json

In [51]:
!rm ../raw_data/map/countries.ndjson
!rm ../raw_data/map/vaccination-by-who-region-2019.ndjson
!rm ../raw_data/map/case-by-who-region-2019.ndjson

## Convert from from geojson to topojson

In [52]:
# npm install -g topojson

!geo2topo ../raw_data/map/countries.json > ../raw_data/map/countries-topo.json

Simplify the topojson to reduce file size

In [53]:
!toposimplify -p 0.01 -f < ../raw_data/map/countries-topo.json > ../raw_data/map/countries-topo-reduced.json

Move into the data file

In [54]:
!mv ../raw_data/map/countries-topo-reduced.json ../src/data/countries-topo-reduced.json